# Section 504.2 - Mixed occupancy
## Building Code
`In a building containing mixed occupancies in accordance with Section 508, no individual occupancy shall exceed the height and number of story limits specified in this section for the applicable occupancies.`

## Individual Occupancy Height Analysis
Each occupancy is checked to determine if the height of the occupancy exceeds the limits defined for the occupancy in Table 504.3. The results are presented in the following table:

In [5]:
import sys
import os

# Add the path to the Rule Notebooks folder to the system path
sys.path.append(os.path.abspath('../..'))

# Import the find_value function from ruleFunctions.py
from ruleFunctions import find_value, check_value, get_building_info

In [2]:
import json

# Load the table from the JSON file
with open('table_504_3.json', 'r') as f:
    table_504_3 = json.load(f)

In [2]:
with open("../../../fuseki_endpoints.txt", "r") as file:
    endpoint_url = file.read()

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(endpoint_url)

In [6]:
# Call the function to get the building information
main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num = get_building_info(endpoint_url)


print(main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num)

Type V A R-1 S 13 4


In [5]:
import pandas as pd

with open("..\..\..\SPARQL\\areasSpaces.txt", "r") as file:
    sparql_query = file.read()

# Set up SPARQL query
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Prepare data for the table
data = {
    "Story": [],
    "Revit ID": [],
    "Occupancy": [],
    "Name": [],
    "Height": [],
    "Allowable Height": [],
    "Height Result": []
}

# Process SPARQL results
for row in results["results"]["bindings"]:
    story = row["levelValue"]["value"]
    revit_id = row["id"]["value"]
    occupancy = str(row["occupancyClassification"]["value"])
    name = row["name"]["value"]
    height = float(row["storeyHeightValue"]["value"])  # Convert height to float
    allowable_height = find_value(table_504_3, occupancy, sprinkler_system, main_construction_type, subtype)
    height_result = check_value(height, allowable_height)

    data["Story"].append(story)
    data["Revit ID"].append(revit_id)
    data["Occupancy"].append(occupancy)
    data["Name"].append(name)
    data["Height"].append(f"{height:.2f} ft")
    data["Allowable Height"].append(allowable_height)
    data["Height Result"].append(height_result)
    
# Convert to DataFrame
df_height = pd.DataFrame(data)

# Check if all rows in the 'Story Result' column are "Passed"
if all(df_height["Height Result"] == "Passed"):
    indv_height_result = "Passed"
else:
    indv_height_result = "Failed"
    
df_height



,Story,Revit ID,Occupancy,Name,Height,Allowable Height,Height Result
0,01,32254,A-3,Ground Floor Gym,0.00 ft,70,Passed
1,01,32254,A-2,Ground Floor Banquet,0.00 ft,70,Passed
2,01,32254,A-2,Ground Floor Breakfast,0.00 ft,70,Passed
3,01,32254,A-3,Ground Floor Pool,0.00 ft,70,Passed
4,01,32254,R-1,Ground Floor Guest,0.00 ft,70,Passed
5,01,32254,B,Ground Floor Business,0.00 ft,70,Passed
6,04,32254,R-1,Fourth Floor,35.33 ft,70,Passed
7,03,32254,R-1,Third Floor,24.33 ft,70,Passed
8,02,32254,R-1,Second Floor,13.33 ft,70,Passed
9,01,32254,B,Ground Floor Public,0.00 ft,70,Passed


## Individual Occupancy Story Analysis
Each occupancy is checked to determine if the story location of the occupancy exceeds the limits defined for the occupancy in Table 504.4. The results are presented in the following table:

In [6]:
# Load the table from the JSON file
with open('table_504_4.json', 'r') as f:
    table_504_4 = json.load(f)

In [7]:
# Prepare data for the table
data = {
    "Story": [],
    "Revit ID": [],
    "Occupancy": [],
    "Name": [],
    "Allowable Story": [],
    "Story Result": []
}

# Process SPARQL results
for row in results["results"]["bindings"]:
    story = str(row["levelValue"]["value"]).lstrip('0')  # Remove leading zeros
    revit_id = row["id"]["value"]
    occupancy = str(row["occupancyClassification"]["value"])
    name = row["name"]["value"]
    allowable_story = find_value(table_504_4, occupancy, sprinkler_system, main_construction_type, subtype)
    story_result = check_value(story, allowable_story)

    data["Story"].append(story)
    data["Revit ID"].append(revit_id)
    data["Occupancy"].append(occupancy)
    data["Name"].append(name)
    data["Allowable Story"].append(allowable_story)
    data["Story Result"].append(story_result)
    
# Convert to DataFrame
df_storey = pd.DataFrame(data)

# Check if all rows in the 'Story Result' column are "Passed"
if all(df_storey["Story Result"] == "Passed"):
    indv_storey_result = "Passed"
else:
    indv_storey_result = "Failed"
    
df_storey

,Story,Revit ID,Occupancy,Name,Allowable Story,Story Result
0,1,32254,A-3,Ground Floor Gym,3,Passed
1,1,32254,A-2,Ground Floor Banquet,3,Passed
2,1,32254,A-2,Ground Floor Breakfast,3,Passed
3,1,32254,A-3,Ground Floor Pool,3,Passed
4,1,32254,R-1,Ground Floor Guest,4,Passed
5,1,32254,B,Ground Floor Business,4,Passed
6,4,32254,R-1,Fourth Floor,4,Passed
7,3,32254,R-1,Third Floor,4,Passed
8,2,32254,R-1,Second Floor,4,Passed
9,1,32254,B,Ground Floor Public,4,Passed


In [8]:
# Check if all results are "Passed"
passed = "Passed"

if indv_height_result != "Passed" or indv_storey_result != "Passed":
    passed = "Failed"

print(f"This section is considered: {passed}\n") 

if (passed == "Passed"): 
    print("No individual occupancy exceeded the height and number of story limits specified in this section for the applicable occupancies.")
else: 
    print("Individual occupancy does exceed height or number of story limits specified in this section for the applicable occupancies.")


This section is considered: Passed

No individual occupancy exceeded the height and number of story limits specified in this section for the applicable occupancies.
